In [1]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import pandas as pd
import pulp
import csv


pd.set_option('display.width', 1000)

In [2]:
# create dataframe of past salaries and past performance
fantasy_df = pd.read_csv("../Data/testdata2.csv")

# Print the first five rows of data to the screen
fantasy_df.head()

,Player_Name,Team,Win,Start,Minutes,FDP,DoubleD,TripleD,FD_Sal,FD_pos,...,bl,to,trey,fg,FGM,FG%,ft,FTM,FT%,ranking
0,Aaron Gordon,orl,Lose,1,34.166667,37.5,0,0,5600,4,...,0,0,4,20-Sep,9,0.45,3-Feb,2,0.666667,Advanced
1,Aaron Holiday,ind,Lose,0,26.683333,24.2,0,0,4100,1,...,0,0,3,11-Jul,7,0.00,0,0,0.000000,AboveAVG
2,Abdel Nader,okc,Win,0,10.866667,4.0,0,0,3500,3,...,1,1,0,3-Jan,1,0.00,0,0,0.000000,belowAVG
3,Adam Mokoka,chi,Lose,0,4.433333,2.4,0,0,3500,2,...,0,0,0,0-2,0,0.00,0,0,0.000000,belowAVG
4,Admiral Schofield,was,Win,0,2.600000,4.2,0,0,3700,3,...,0,0,0,0,0,0.00,0,0,0.000000,belowAVG


In [3]:
fantasy_df['FD_pos'] = fantasy_df['FD_pos'].replace([1,2,3,4,5],['PG', 'SG', 'SF', 'PF', 'C'])
fantasy_df['Player_Name'] = fantasy_df['Player_Name'].replace(' ', '_', regex=True)
fantasy_df

,Player_Name,Team,Win,Start,Minutes,FDP,DoubleD,TripleD,FD_Sal,FD_pos,...,bl,to,trey,fg,FGM,FG%,ft,FTM,FT%,ranking
0,Aaron_Gordon,orl,Lose,1,34.166667,37.5,0,0,5600,PF,...,0,0,4,20-Sep,9,0.450000,3-Feb,2,0.666667,Advanced
1,Aaron_Holiday,ind,Lose,0,26.683333,24.2,0,0,4100,PG,...,0,0,3,11-Jul,7,0.000000,0,0,0.000000,AboveAVG
2,Abdel_Nader,okc,Win,0,10.866667,4.0,0,0,3500,SF,...,1,1,0,3-Jan,1,0.000000,0,0,0.000000,belowAVG
3,Adam_Mokoka,chi,Lose,0,4.433333,2.4,0,0,3500,SG,...,0,0,0,0-2,0,0.000000,0,0,0.000000,belowAVG
4,Admiral_Schofield,was,Win,0,2.600000,4.2,0,0,3700,SF,...,0,0,0,0,0,0.000000,0,0,0.000000,belowAVG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,Zach_LaVine,chi,Lose,1,33.433333,42.9,0,0,8100,SG,...,0,3,0,10-Jun,6,0.600000,8-Jun,6,0.750000,Advanced
494,Zach_Norvell_Jr.,lal,Win,0,2.333333,0.0,0,0,3500,SG,...,0,0,0,0,0,0.000000,0,0,0.000000,belowAVG
495,Zhaire_Smith,phi,Lose,0,3.266667,0.0,0,0,3500,SG,...,0,0,0,0-1,0,0.000000,0,0,0.000000,belowAVG
496,Zion_Williamson,nor,Lose,1,32.983333,37.5,1,0,6500,PF,...,1,3,0,14-Aug,8,0.571429,7-May,5,0.714286,Advanced


In [4]:
fantasy_df.corr()['FDP']

Start      0.573787
Minutes    0.829667
FDP        1.000000
DoubleD    0.514324
TripleD    0.208441
FD_Sal     0.684451
ADI        0.045966
VMI       -0.004520
pt         0.885927
rb         0.742692
as         0.651706
st         0.494994
bl         0.427478
to         0.460502
trey       0.513848
FGM        0.641857
FG%        0.368892
FTM        0.534222
FT%        0.421539
Name: FDP, dtype: float64

In [5]:
to_drop = ['TripleD', 'ADI', 'VMI', 'FG%', 'FT%']
fantasy_df.drop(to_drop, axis=1, inplace = True)
fantasy_df

,Player_Name,Team,Win,Start,Minutes,FDP,DoubleD,FD_Sal,FD_pos,pt,...,as,st,bl,to,trey,fg,FGM,ft,FTM,ranking
0,Aaron_Gordon,orl,Lose,1,34.166667,37.5,0,5600,PF,24,...,1,2,0,0,4,20-Sep,9,3-Feb,2,Advanced
1,Aaron_Holiday,ind,Lose,0,26.683333,24.2,0,4100,PG,17,...,4,0,0,0,3,11-Jul,7,0,0,AboveAVG
2,Abdel_Nader,okc,Win,0,10.866667,4.0,0,3500,SF,2,...,0,0,1,1,0,3-Jan,1,0,0,belowAVG
3,Adam_Mokoka,chi,Lose,0,4.433333,2.4,0,3500,SG,0,...,0,0,0,0,0,0-2,0,0,0,belowAVG
4,Admiral_Schofield,was,Win,0,2.600000,4.2,0,3700,SF,0,...,0,1,0,0,0,0,0,0,0,belowAVG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,Zach_LaVine,chi,Lose,1,33.433333,42.9,0,8100,SG,18,...,7,3,0,3,0,10-Jun,6,8-Jun,6,Advanced
494,Zach_Norvell_Jr.,lal,Win,0,2.333333,0.0,0,3500,SG,0,...,0,0,0,0,0,0,0,0,0,belowAVG
495,Zhaire_Smith,phi,Lose,0,3.266667,0.0,0,3500,SG,0,...,0,0,0,0,0,0-1,0,0,0,belowAVG
496,Zion_Williamson,nor,Lose,1,32.983333,37.5,1,6500,PF,21,...,1,1,1,3,0,14-Aug,8,7-May,5,Advanced


# LINEAR MODELING

In [7]:
# formulate linear programming problem
problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)

#Defining Variables

In [8]:
# initialize variables
playerPositions = fantasy_df['FD_pos'].tolist()
playerNames = fantasy_df['Player_Name'].tolist()
playerCosts = tuple(fantasy_df['FD_Sal'])
playerPoints = tuple(fantasy_df['FDP'])
salaryCap = 60000
numOfPlayers = range(len(playerNames))

In [9]:
# create variable to represent each player
playerInLineup = pulp.LpVariable.dicts("Player", [i for i in numOfPlayers], 0, 1, cat="Binary")


#Creating the Objective(s)

In [10]:
# Objective: maximize sum of player points in FanDuel
problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)
print(problem)

Optimal_Lineup:
MAXIMIZE
37.5*Player_0 + 24.2*Player_1 + 12.9*Player_10 + 34.7*Player_100 + 43.8*Player_102 + 13.1*Player_103 + 21.4*Player_104 + 33.7*Player_105 + 12.7*Player_106 + 34.0*Player_107 + 26.5*Player_108 + 39.4*Player_109 + 7.4*Player_11 + 2.2*Player_110 + 41.8*Player_111 + 16.6*Player_112 + 6.2*Player_113 + 20.7*Player_115 + 24.9*Player_116 + 5.2*Player_117 + 2.5*Player_118 + 18.5*Player_119 + 1.0*Player_12 + 44.4*Player_120 + 23.7*Player_122 + 45.7*Player_124 + 24.6*Player_125 + 16.0*Player_126 + 34.6*Player_127 + 34.4*Player_128 + 22.9*Player_129 + 0.2*Player_13 + 45.4*Player_130 + 10.7*Player_131 + 6.9*Player_132 + 50.4*Player_133 + 2.0*Player_134 + 13.7*Player_135 + 2.2*Player_136 + 14.8*Player_137 + 5.1*Player_138 + 2.4*Player_14 + 8.8*Player_140 + 3.2*Player_141 + 43.9*Player_142 + 10.2*Player_143 + 0.5*Player_144 + 28.1*Player_145 + 28.4*Player_146 + 25.2*Player_147 + 14.4*Player_148 + 20.4*Player_149 + 3.0*Player_15 + 11.8*Player_150 + 12.0*Player_151 + 22.2*Player

# Defining the Contraints

In [11]:
# constraint n1: each player can only be chosen at most once
for i in numOfPlayers:
    problem += pulp.lpSum(playerInLineup[i]) <= 1
    

In [12]:
# constraint n2: sum of player costs cannot be more than or equal to FanDuel Salary cap ($ 60,000)
problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap

In [13]:
# constraint n3: teams can have up to 9 players
problem += pulp.lpSum(problem.variables()) == 9

#Player Positions

In [14]:
# constraint n4: number of PG = 2
pointguards = tuple(fantasy_df['FD_pos'] == 'PG')
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) >=1
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) <= 2

In [15]:
# constraint n5: number of SG = 2
shootinguard = tuple(fantasy_df['FD_pos'] == 'SG')
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) <= 2

In [16]:
# constraint n6: number of SF = 2
smallforward = tuple(fantasy_df['FD_pos'] == 'SF')
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) <= 2

In [17]:
# constraint n7: number of PF = 2
powerforward = tuple(fantasy_df['FD_pos'] == 'PF')
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) <= 2

In [18]:
# constraint n8: number of C = 1
center = tuple(fantasy_df['FD_pos'] == 'C')
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) >=1
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) <= 1

# Solving the Problem

In [22]:
# calculating predicted lineup
cost = 0
points = 0

predicted = set()
if problem.solve() == 1:
    for pos in range(len(numOfPlayers)):
        if playerInLineup[pos].value() == 1:
            name = playerNames[pos]
            predicted.add(name)
            cost += playerCosts[pos]
            points += playerPoints[pos]
            print (playerNames[pos], playerPositions[pos], playerCosts[pos],playerPoints[pos])
    print ('-------------------')
    print (f"The total cost is =: {cost}")
    print (f"The predicted FanDuel points: {points}")
else:
    print ('Error finding solution')

Bradley_Beal SG 9600 60.2
Bruce_Brown SG 4300 43.0
Damian_Lillard PG 10200 73.4
Dorian_Finney-Smith SF 4500 45.4
Draymond_Green PF 5800 50.4
Kristaps_Porzingis PF 6400 54.4
Miles_Bridges SF 4800 39.2
Monte_Morris PG 4300 40.9
Nikola_Jokic C 10000 80.5
-------------------
The total cost is =: 59900
The predicted FanDuel points: 487.4
